In [ ]:
import pandas as pd
import numpy as np

num_agrupado = ["10588","10613","10613","10619","10622","10632","10632","10634","10634","10652","10652","10667","10667","10516","10528","10528","10554","10605","10637","10637","10637","10644","10644","10644","10644","10488","10488","10509","10509","10556","10556","10557","10557","10594","10594","10595","10595","10679","10679","10680","10680","10748","10748","10761","10761","10426","10426","10426","10433","10433","10433","10433","10482","10482","10482","10482","10484","10484","10484","10492","10492","10500","10507","10521","10527","10527","10531","10531"]

Radares = [5415,5483,5126,5471,5470,5316,5315,5313,5314,5358,5357,5355,5356,5463,5297,5296,5354,5098,5476,5475,5474,5479,5478,5477,5480,5323,5226,5308,5307,5432,5433,5414,5413,5201,5200,5199,5198,5205,5204,5203,5202,5196,5197,5195,5194,5445,5104,5446,5103,5462,5461,5460,5441,5106,5443,5442,5439,5105,5440,5457,5449,5317,5101,5506,5336,5335,5402,5401]


# cria um dicionário
mapa = {}
for i in range(len(num_agrupado)):
    mapa[Radares[i]] = num_agrupado[i]

In [ ]:
pd.set_option('display.max_columns', None)
a = pd.read_csv(r'D:\Users\guilh\Documents\GitHub\Dados_CET\csv_radares-20200426T211130Z-001_selecionados/'+"01_selecionado.csv", index_col=["Unnamed: 0"])
b = a.sample(frac=0.1)
a.head()

In [ ]:
# verificação
# antes temos um x registros no (radar, faixa) = (5479,3)
# e y registros (radar, faixa) = (5478,_) sendo _ qualquer faixa


display(a.loc[(a["Local"]==5479) & (a["Faixa"] == 3), ["Local", "Faixa"]])
display(a.loc[(a["Local"]==5478), ["Local", "Faixa"]])
# mas lembrando que ainda tiramos os nan

In [ ]:
# cria a coluna de numeros agrupados
a['Numero Agrupado'] = a['Local'].map(mapa)

#coloca a Data e datetime object para facilitar manipulacao
a.Data=pd.to_datetime(a.Data)

# cria a coluna do dia da semana
# ###############IMPORTANTE############# segunda = 0
a["dia_sem"] = a.Data.dt.dayofweek

# limpa a ocupacao
a.drop(a.loc[a['Ocupacao'] =='     ','Ocupacao'].index, inplace=True)
a.Ocupacao = a.Ocupacao.astype(int)

#dropa vazios
a.drop(a.loc[a['Velocidade'] =='   ','Velocidade'].index, inplace=True)

#dropa não válidos (nan)
a.loc[:,'Velocidade'].dropna(inplace=True)

#coloca tudo no mesmo formato de dado int
a.loc[:,'Velocidade'] = pd.to_numeric(a.loc[:,'Velocidade'])

########## UPDATE ###########
# transforma todos radares 5479 na faixa 3, (5479,3) em radares 5478 na faixa 3 (5478,3)
# pode ficar tranquilo por que eles tem o mesmo numero agrupado
a.loc[(a["Local"]==5479) & (a["Faixa"] == 3), "Local"] = 5478


a.reset_index(drop=True,inplace=True)


a = a[['Lote', 'Data', "dia_sem", 'Local', 'Numero Agrupado', 'Faixa', 'Entrefaixa',
       'Registro', 'Tipo de Registro', 'Especie',
      'Classe', 'Comprimento', 'Velocidade', 'Ocupacao']]
a.head()

In [ ]:
## verificao 
# nao devem mais haver (5479,3)
# e assim x deve ser adicionado a y

display(a.loc[(a["Local"]==5479) & (a["Faixa"] == 3), ["Local", "Faixa"]])


In [ ]:
Ag = a

Ag.loc[:,'V1'] = Ag.loc[:,'Velocidade'].copy()
Ag.loc[:,'V2'] = Ag.loc[:,'Velocidade'].copy()
Ag.loc[:,'Oc1'] = Ag.loc[:,'Ocupacao'].copy()
Ag.loc[:,'Oc2'] = Ag.loc[:,'Ocupacao'].copy()
Ag = Ag.groupby([ 'Numero Agrupado',"Faixa",'Especie',pd.Grouper(key='Data', freq='5Min')]).agg({'dia_sem': "mean",
                                                                        "Registro": "count",
                                                                        'Velocidade': 'mean',
                                                                        'V1':'median',
                                                                        'V2':'std',
                                                                        'Ocupacao':'mean',
                                                                        'Oc1':'median',
                                                                        'Oc2':'std',
                                                                        })
Ag.columns=['dia_sem',
            "Volume",
            'Vel_media',
            'Vel_mediana',
            'Vel_desvpad',
            'Ocu_media',
            'Ocu_mediana',
            'Ocu_desvpad',
            ] # Renomeia

    

 #   df_group = df_new_s.groupby(['Local','Número Agrupado','Especie','Classe', pd.Grouper(key='Data', freq=freq)]).agg({
#                                                                            'Velocidade': 'mean',
##                                                                            'V1':'median',
 #                                                                           'V2':'std',
#                                                                            'Registro':'count'})
Ag

In [ ]:
Ag.reset_index(inplace=True)
c = Ag
Ag.head()

In [ ]:
c["Periodo"] = "y"

c.head()

In [ ]:

labels = np.array(["P1","P2","P3","P4","P5","P6","P7"])
intervalos = pd.IntervalIndex.from_tuples([(0,5),(5,7),(7,10),(10,14),(14,17),(17,20),(20,24)], closed="left")

c['aux'] = pd.cut(np.array(c.Data.dt.hour), intervalos)
# if c.loc[  ( c['Data'].dt.hour > 0) and (c['Data'].dt.hour <= 5)  ] :

#     c["Periodo"] = "P1"
display(c)

In [ ]:
intervalos = pd.IntervalIndex.from_tuples([(0,5),(5,7),(7,10),(10,14),(14,17),(17,20),(20,24)])
# inter = ["(0.0, 5.0]","(5.0, 7.0]","(7.0, 10.0]","(10.0, 14.0]","(14.0, 17.0]","(17.0, 20.0]","(20.0, 24.0]"]

sentidos = ["Sumare_N","Sumare_S","Sumare_S","Sumare_S","Sumare_N","Sumare_N","Sumare_N","Sumare_S","Sumare_S","Sumare_S","Sumare_S","Sumare_N","Sumare_N","Brasil_N","Brasil_S","Brasil_S","Brasil_S","Brasil_S","Brasil_S","Brasil_S","Brasil_S","Brasil_N","Brasil_N","Brasil_N","Brasil_N","Reboucas_C","Reboucas_C","Reboucas_B","Reboucas_B","Reboucas_B","Reboucas_B","Reboucas_C","Reboucas_C","Reboucas_B","Reboucas_B","Reboucas_C","Reboucas_C","Reboucas_B","Reboucas_B","Reboucas_C","Reboucas_C","Reboucas_B","Reboucas_B","Reboucas_C","Reboucas_C","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_C","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_C","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_B","23deMaio_C","23deMaio_C"]




inter = ["[0, 5)","[5, 7)","[7, 10)","[10, 14)","[14, 17)","[17, 20)","[20, 24)"]
# "P1","P2","P3","P4","P5","P6","P7"

mapb = {}
for i in range(len(labels)):
    mapb[inter[i]] = labels[i]


try:
    c['aux'] = c['aux'].astype(str)
    c['Periodo'] = c['aux'].map(mapb)
    c.drop(columns=["aux"],inplace=True, axis = 0)
except:
    pass


c["NumAg_fx"] = c["Numero Agrupado"].astype(str) + c['Faixa'].astype(str)


mapc = {}
for i in range(len(num_agrupado)):
    mapc[num_agrupado[i]] = sentidos[i]

c["Eixo"] = c["Numero Agrupado"].map(mapc)



# display(c[["Numero Agrupado", "Data", "Especie", "Periodo"]])
c =c[["Data", "dia_sem", "Periodo", "Numero Agrupado", "Faixa", "NumAg_fx","Eixo","Especie","Volume", "Vel_media", "Vel_mediana", "Vel_desvpad", "Ocu_media", "Ocu_mediana", "Ocu_desvpad"]] 
display(c.head())

In [ ]:
colunas = ["Id","Data", "dia_sem", "Periodo", "Numero Agrupado", "Faixa", "Fx_tipo", "NumAg_fx", "eixo","Especie","Volume", "Vel_media", "Vel_mediana", "Vel_desvpad", "Oc_media", "Oc_mediana", "Oc_desvpad", "chuva_5min", "chuva_1h"]
len(colunas)

In [ ]:
tip_faixa = ["mista","mista","mista","onibus","mista","mista","mista","mista","onibus","mista","mista","mista","mista","onibus","mista","mista","mista","mista","onibus","onibus","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","onibus","mista","mista","mista","mista","mista","mista","onibus","mista","mista","onibus","mista","mista","mista","onibus","mista","mista","mista","onibus","mista","mista","mista","onibus","mista","mista","mista","onibus","mista","mista","mista","mista","mista","mista","mista","onibus","mista","mista","onibus","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","onibus","mista","mista","onibus","mista","mista","mista","mista","mista","onibus","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","mista","onibus","mista","mista","mista","onibus","onibus","mista","mista","onibus","mista","mista","onibus","mista","mista","onibus","mista","mista"]

num_faixa = ["104261","104262","104263","104264","104331","104332","104333","104334","104335","104821","104822","104823","104824","104825","104841","104842","104843","104844","104845","104881","104882","104883","104884","104921","104922","104923","105001","105002","105003","105071","105072","105091","105092","105093","105094","105161","105162","105163","105164","105211","105212","105213","105281","105282","105283","105284","105311","105312","105313","105314","105541","105542","105543","105561","105562","105563","105564","105571","105572","105573","105574","105881","105882","105883","105884","105941","105942","105943","105951","105952","105953","106051","106052","106053","106131","106132","106133","106134","106135","106191","106192","106193","106221","106222","106223","106321","106322","106323","106341","106342","106343","106371","106372","106373","106374","106375","106376","106441","106442","106443","106444","106445","106446","106521","106522","106523","106524","106671","106672","106673","106674","106791","106792","106793","106801","106802","106803","107481","107482","107483","107611","107612","107613"]

mapfaixas = {}
for i in range(len(tip_faixa)):
    mapfaixas[num_faixa[i]] = tip_faixa[i]

c["Fx_tipo"] = c["NumAg_fx"].map(mapfaixas)

c =c[["Data", "dia_sem", "Periodo", "Numero Agrupado", "Faixa", "NumAg_fx","Fx_tipo","Eixo","Especie","Volume", "Vel_media", "Vel_mediana", "Vel_desvpad", "Ocu_media", "Ocu_mediana", "Ocu_desvpad"]] 
display(c)

# caso queria fazer alguma verificação mais específica
# c.loc[  (c["Numero Agrupado"] == "10556") & (c["Faixa"] == 2)  ]